# A simple function

![](img/Fibonacci_staircase_at_St_Johns_College_Cambridge_UK_Valerian_Guillot.jpg)

In [ ]:
def fib(n):
    if n <= 1:
        return n
    else:
        return fib(n - 2) + fib(n - 1)

## ...can be tested interactively

In [ ]:
fib(5)

## Looks about right, ship it.

# But it could `assert` itself...

In [ ]:
def fib(n):
    """ Return the `n`th fibonacci sequence number
    >>> assert fib(5) == 5
    """
    if n <= 1:
        return n
    else:
        return fib(n - 2) + fib(n - 1)


print(fib(2))
print(fib(10))

## ... and be **unit** tested with `doctest`


In [ ]:
import doctest
doctest.testmod(verbose=True)

# **type hints** add intent

In [ ]:
def fib_typed(n: int) -> int:
    if n <= 1:
        return n
    else:
        return fib_typed(n - 2) + fib_typed(n - 1)

## ... but only do so much alone

In [ ]:
try:
    fib_typed("foo")  # actually have to enter the code
except Exception as err:
    print(err)

# **type check** with `beartype`

In [ ]:
from beartype import beartype

@beartype
def fib_bear(n: int) -> int:
    if n <= 1:
        return n
    else:
        return fib_bear(n - 2) + fib_bear(n - 1)

In [ ]:
try:
    fib_bear("bear")  # rejects bad inputs at the door
except Exception as err:
    print(err)

## ...but what does it **cost**?

# **benchmark** with `timeit`

In [ ]:
t0 = %timeit -o fib(10)

In [ ]:
t1 = %timeit -o fib_typed(10)

In [ ]:
t2 = %timeit -o fib_bear(10)

## ... are these **type hints** really worth it?

# **compile** with `mypyc`

In [ ]:
%reload_ext mypyc_ipython

In [ ]:
%%mypyc
def fib_c(n: int) -> int:
    if n <= 1:
        return n
    else:
        return fib_c(n - 2) + fib_c(n - 1)

In [ ]:
t3 = %timeit -o fib_c(10)

In [ ]:
[
    f"{int(t.average / t3.average)}x"
    for t in [t0, t1, t2]
]

# **profile** with `snakeviz` 

In [ ]:
%%file fib.py
from beartype import beartype

def fib_typed(n: int) -> int:
    if n <= 1:
        return n
    else:
        return fib_typed(n - 2) + fib_typed(n - 1)

@beartype
def fib_bear(n: int) -> int:
    if n <= 1:
        return n
    else:
        return fib_bear(n - 2) + fib_bear(n - 1)

In [ ]:
%reload_ext snakeviz
%snakeviz import fib; fib.fib_typed(20); fib.fib_bear(20)

# **fuzz** test with `atheris`

In [ ]:
%%file fuzz.py
import atheris

with atheris.instrument_imports():
    import fib
    import sys

def TestOneInput(data):
    fdp = atheris.FuzzedDataProvider(data)
    fib.fib_typed(fdp.ConsumeInt(1))

atheris.Setup(sys.argv, TestOneInput)
atheris.Fuzz()

In [ ]:
!coverage run --branch --source fib fuzz.py -atheris_runs=3
!coverage html

# Up the stack

[A simple web app](./02-app.ipynb)